# Synthesis

Classiq synthesis engine takes a high-level program written in Classiq's Qmod language, and compiles it into an executable gate-based circuit.\
In most cases, there is not a single implementation that is superior in all manners - there might be one implementation that requires the lowest number of qubits, another that minimizes the gate-count etc. 

Let's look at a simple model, and use Classiq synthesis engine to compile it according to different optimization parameters.

In [6]:
from classiq import *

@qfunc
def main(x: Output[QNum], y: Output[QNum]) -> None:
    allocate(3,x)
    hadamard_transform(x)
    y |= x**2 + 1

First, let's synthesize to optimize circuit depth, i.e. to minimize longest chain of gates in the circuit.

In [7]:
qmod_opt_depth = create_model(
    entry_point=main,
    constraints=Constraints(optimization_parameter=OptimizationParameter.DEPTH) 
)

qprog_opt_depth = synthesize(qmod_opt_depth)

We can inspect the resulting curcuit using Classiq's web visualization:


In [8]:
show(qprog_opt_depth)

Opening: https://platform.classiq.io/circuit/2t7ALaL8G99fehfqWPbX4pjBim5?version=0.68.0


On the left side menu, under 'Program info', you should see the resulting depth, width and gate-count:

<div style="text-align:center;">
    <img src="https://docs.classiq.io/resources/qprog_opt_depth.png " alt="vis" style="width: 400px; height: auto;"/>
</div>

The resulting depth and width are 42 and 16, respectively.

Now, let's synthesize to optimize width, i.e to minimize the number of qubits used.\
We can do it by calling `set_constraints`, which overrides the current `constraints` parameter:

In [9]:
qmod_opt_width = set_constraints(
    serialized_model=qmod_opt_depth,
    constraints=Constraints(optimization_parameter=OptimizationParameter.WIDTH)
)

qprog_opt_widht = synthesize(qmod_opt_width)

Inspect the resulting circuit:

In [10]:
show(qprog_opt_widht)

Opening: https://platform.classiq.io/circuit/2t7AMF1Pqypoa1FfklsGHOz6E2Z?version=0.68.0


The new depth and width are 47 and 9, respectively. As expected, we "pay" with extra depth for an implementation that uses less qubits.

### Excercise

Create your own `main` function and synthesize it with different constraints.\
Note that `OptimizationParameter` is only one kind of configuration possible. Classiq synthesis engine also supports rigid constraints of `max_width`, `max_depth` and `max_gate_count`. 

